In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers

#### Reading the PDF data

In [ ]:
# Loading the data from PDF file
def data_extractor(dir_path):
    loader = DirectoryLoader(dir_path, # directory path
                            glob="*.pdf", # only pdf files
                            loader_cls = PyPDFLoader, # using module
                            show_progress=True,
                            use_multithreading=True
                            )
    # loading the pdf documents
    documents = loader.load()

    return documents

In [ ]:
pdf_data = data_extractor("D:\Projects\Medical-Chat-Bot-using-Llama-2\data\\")
# print(pdf_data)

#### Splitting corpus into text chunks

In [ ]:
# Create text chunks
def text_splitter(document):
    text_chunks = RecursiveCharacterTextSplitter(chunk_size=500,
                                    chunk_overlap = 20
                                )
    return text_chunks.Create_documents(document)

In [ ]:
text_chunks = text_splitter(pdf_data)
print('Length of text chunks : ',len(text_chunks))

In [ ]:
text_chunks

In [ ]:
text_chunks[524].page_content

#### Converting text to vectors

In [ ]:
# Downloading the embedding model from hugging face
EMBD_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = HuggingFaceEmbeddings(model_name=EMBD_MODEL_NAME)

In [ ]:
from dotenv import load_dotenv
import os

PINE_CONE_API_KEY = os.environ.get('PINE_CONE_API_KEY')
PINE_CONE_ENV = os.environ.get('PINE_CONE_ENV')

In [ ]:
# initializing the vector DB pinecone
pinecone.init(api_key=PINE_CONE_API_KEY,
              environment=PINE_CONE_ENV
            )
index_name = "medical-chat-bot-llama-2"

In [ ]:

# creating embeddings for each text chunk and storing into pinecone
doc_search = Pinecone.from_texts([i.page_content for i in text_chunks], # list of chunks text
                                  embedding_model, # Embedding model 
                                  index_name = index_name  # pinecone index name
                                  )

In [ ]:
# loading the existing index from pinecone
doc_search = Pinecone.from_existing_index(index_name,embedding_model)

In [ ]:
# model_type='llama',
model = CTransformers(model="TheBloke/Llama-2-7B-Chat-GGML",
                    model_file='llama-2-7b-chat.ggmlv3.q5_0.bin',
                    device_map='cuda',
                    config = {'temperature':0.35,
                            'max_new_tokens':1024,
                            # 'repetition_penalty':0.7,
                            'stream':True,
                            # 'context_length':-1
                            })

In [ ]:
# loading the prompt
prompt_template="""
                Use the following pieces of information to answer the user's question.
                If you don't know the answer, just say that you don't know, don't try to make up an answer.

                Context: {context}
                Question: {question}

                Only return the helpful answer below and nothing else.
                Helpful answer:
                """
prompt = PromptTemplate(template=prompt_template,
                        input_variables=['context','question'],
                        )
chain_type_kwargs = {'prompt':prompt}

In [ ]:
qa_bot = RetrievalQA.from_chain_type(llm=model,
                                        chain_type="stuff",
                                        retriever = doc_search.as_retriever(search_kwargs={'k':4}),
                                        return_source_documents=True,
                                        chain_type_kwargs=chain_type_kwargs
                                        )

In [ ]:
qa_bot({"query": 'Hello'})